#### donut-base
- swintransformer + bart-base-decoder
#### donut-large
- swintransformer + bart-larger-decoder

In [ ]:
MP1 = "J:/model/mllm-model/donut-pretrain/20240102/pl-checkpoint-159500-ned-0.8607632262639043"
MP2 = "J:/model/facebook_bart-large"

In [2]:
import sys
from transformers import (VisionEncoderDecoderModel,
                          VisionEncoderDecoderConfig,
                          DonutProcessor,
                          AutoProcessor,
                          BartModel,
                          MBartModel,
                          BartConfig,
                          MBartConfig,
                          AutoTokenizer)
sys.path.append("../../src/multimodallm")

In [3]:
from multimodallm.utils import cal_model_parameters

In [4]:
donut_base_config = VisionEncoderDecoderConfig.from_pretrained(MP1)
donut_base_model = VisionEncoderDecoderModel.from_pretrained(MP1)

In [5]:
donut_base_params = cal_model_parameters(donut_base_model)
donut_base_encoder_params = cal_model_parameters(donut_base_model.encoder)
donut_base_decoder_params = cal_model_parameters(donut_base_model.decoder)
donut_base_params, donut_base_encoder_params, donut_base_decoder_params

(204299384, 74180728, 130118656)

In [6]:
bart_large_config = MBartConfig.from_pretrained(MP2)
bart_large_model = MBartModel.from_pretrained(MP2)
bart_large_tokenizer = AutoTokenizer.from_pretrained(MP2)

You are using a model of type bart to instantiate a model of type mbart. This is not supported for all configurations of models and can yield errors.
You are using a model of type bart to instantiate a model of type mbart. This is not supported for all configurations of models and can yield errors.
Some weights of MBartModel were not initialized from the model checkpoint at J:/model/facebook_bart-large and are newly initialized: ['decoder.layer_norm.weight', 'encoder.layer_norm.bias', 'encoder.layer_norm.weight', 'decoder.layer_norm.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
bart_large_params = cal_model_parameters(bart_large_model)
bart_large_encoder_params = cal_model_parameters(bart_large_model.encoder)
bart_large_decoder_params = cal_model_parameters(bart_large_model.decoder)
bart_large_params, bart_large_encoder_params, bart_large_decoder_params

(406295552, 203680768, 254086144)

In [8]:
inputs = bart_large_tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = bart_large_model(**inputs)
outputs

Seq2SeqModelOutput(last_hidden_state=tensor([[[ 1.8731e-01,  1.2451e+00, -2.6025e-02,  ...,  2.7185e-01,
          -7.0134e-02,  2.0836e-01],
         [ 1.8448e-01,  1.3049e+00, -3.7771e-02,  ...,  2.7551e-01,
          -6.2602e-02,  2.0327e-01],
         [ 1.8678e-01,  9.1669e-01, -3.1941e-02,  ...,  2.6436e-01,
          -5.6333e-02,  2.2700e-01],
         ...,
         [ 1.8920e-01,  2.1786e-01, -1.0730e-01,  ...,  3.3643e-01,
          -8.3321e-02,  2.8835e-01],
         [ 1.4821e-01,  5.8140e-01, -1.1566e-01,  ...,  3.2180e-01,
          -6.0410e-02,  2.1415e-01],
         [ 2.2279e-01,  1.2014e-03, -1.9514e-01,  ...,  3.6329e-01,
          -6.7464e-02,  2.9462e-01]]], grad_fn=<NativeLayerNormBackward0>), past_key_values=((tensor([[[[ 6.4695e+00,  7.4873e+00,  1.3615e+01,  ...,  5.5435e+00,
            1.9106e+01, -4.8808e+00],
          [ 2.4149e+00,  6.9186e+00,  9.8275e+00,  ...,  4.7713e+00,
            1.3457e+01, -6.1739e+00],
          [-1.1826e+00, -1.9506e+00, -6.1460e+00

### donut-large

In [9]:
donut_large_config = VisionEncoderDecoderConfig.from_encoder_decoder_configs(
    donut_base_model.encoder.config, bart_large_model.decoder.config
)
donut_large_model = VisionEncoderDecoderModel(
    donut_large_config, donut_base_model.encoder, bart_large_model.decoder
)

In [10]:
donut_large_model_params = cal_model_parameters(donut_large_model)
donut_large_model, donut_large_model_params

(VisionEncoderDecoderModel(
   (encoder): DonutSwinModel(
     (embeddings): DonutSwinEmbeddings(
       (patch_embeddings): DonutSwinPatchEmbeddings(
         (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
       )
       (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
       (dropout): Dropout(p=0.0, inplace=False)
     )
     (encoder): DonutSwinEncoder(
       (layers): ModuleList(
         (0): DonutSwinStage(
           (blocks): ModuleList(
             (0-1): 2 x DonutSwinLayer(
               (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
               (attention): DonutSwinAttention(
                 (self): DonutSwinSelfAttention(
                   (query): Linear(in_features=128, out_features=128, bias=True)
                   (key): Linear(in_features=128, out_features=128, bias=True)
                   (value): Linear(in_features=128, out_features=128, bias=True)
                   (dropout): Dropout(p=0.0, inpla

In [11]:
SAVE_MP = "J:/model/mllm-model/donut-large"

In [12]:
processor = DonutProcessor.from_pretrained(MP1)
donut_large_model.config.pad_token_id = processor.tokenizer.pad_token_id
donut_large_model.config.decoder_start_token_id = processor.tokenizer.convert_tokens_to_ids(["<s>"])[0]

donut_large_model.config.pad_token_id, donut_large_model.config.decoder_start_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


(1, 0)

In [13]:
expand_vocab = ["</n>"]
newly_added_num = processor.tokenizer.add_tokens(expand_vocab)

# if newly_added_num > 0:
donut_large_model.decoder.resize_token_embeddings(len(processor.tokenizer))

max_length = 2560
if max_length > donut_large_config.decoder.max_position_embeddings:
    donut_large_config.decoder.max_position_embeddings = max_length
    ignore_mismatched_sizes = True

In [14]:
donut_large_model.save_pretrained(SAVE_MP)
processor.save_pretrained(SAVE_MP)

In [15]:
donut_large_model_load = VisionEncoderDecoderModel.from_pretrained(SAVE_MP)

Config of the decoder: <class 'transformers.models.mbart.modeling_mbart.MBartForCausalLM'> is overwritten by shared decoder config: MBartConfig {
  "_name_or_path": "J:/model/facebook_bart-large",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_cross_attention": true,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2

In [16]:
newly_added_num

0

In [17]:
donut_base_model.decoder.model.decoder

MBartDecoder(
  (embed_tokens): Embedding(58891, 1024, padding_idx=1)
  (embed_positions): MBartLearnedPositionalEmbedding(2562, 1024)
  (layers): ModuleList(
    (0-3): 4 x MBartDecoderLayer(
      (self_attn): MBartAttention(
        (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
      )
      (activation_fn): GELUActivation()
      (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder_attn): MBartAttention(
        (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
      )
     

In [18]:
donut_large_model_load.decoder, donut_base_model.decoder

(MBartForCausalLM(
   (model): MBartDecoderWrapper(
     (decoder): MBartDecoder(
       (embed_tokens): Embedding(58891, 1024, padding_idx=1)
       (embed_positions): MBartLearnedPositionalEmbedding(2562, 1024)
       (layers): ModuleList(
         (0-11): 12 x MBartDecoderLayer(
           (self_attn): MBartAttention(
             (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
             (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
             (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
             (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
           )
           (activation_fn): GELUActivation()
           (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
           (encoder_attn): MBartAttention(
             (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
             (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
             (q_

In [19]:
donut_large_decoder_tokens_embedding_params = cal_model_parameters(donut_large_model_load.decoder.model.decoder.embed_tokens)

donut_large_decoder_positions_embedding_params = cal_model_parameters(donut_large_model_load.decoder.model.decoder.embed_positions)

donut_large_decoder_lm_head_params = cal_model_parameters(donut_large_model_load.decoder.lm_head)

donut_large_decoder_tokens_embedding_params, donut_large_decoder_positions_embedding_params, donut_large_decoder_lm_head_params

(60304384, 2623488, 60304384)

In [20]:
donut_base_token_embedding_params_rate = donut_large_decoder_tokens_embedding_params / donut_base_params
donut_large_token_embedding_params_rate = donut_large_decoder_tokens_embedding_params / donut_large_model_params
donut_base_token_embedding_params_rate, donut_large_token_embedding_params_rate

(0.2951765336698225, 0.18370536031427503)

- donut模型使用MBart作为解码器, token数量58891, 使得token_embedding和lm_head层的参数量非常大, donut-base模型参数量占比接近60%, donut-large模型参数量占比约36%
- 下一步需要减少token数,并重构token_embedding和lm_head层的参数矩阵, 确保现有模型效果不影响

#### 区分tokenizer里面的语言

In [21]:
import langid

In [22]:
res = langid.classify("what is your name")
res

('en', -33.54452037811279)

In [23]:
processor.tokenizer.vocab

{'wear': 54272,
 'ことに': 41572,
 'مجلس': 4644,
 '▁From': 35067,
 'cula': 15682,
 '▁cere': 40064,
 'pis': 42969,
 'EC': 15265,
 '▁•': 51233,
 '▁враг': 57238,
 'cidas': 48839,
 '▁клінічн': 21057,
 '근': 27265,
 'gner': 43346,
 '有不少': 3017,
 '▁one': 14754,
 '扩张': 19540,
 '▁Outlook': 48498,
 '▁Selv': 22546,
 '创造': 36471,
 '▁recommend': 48440,
 'netz': 7987,
 'ultima': 51582,
 '▁команду': 8070,
 '▁nagyjából': 23409,
 'tol': 9134,
 'jia': 46791,
 '▁Пир': 6190,
 'jä': 49278,
 '464': 14277,
 '争议': 8041,
 '▁gedink': 24394,
 '救援': 1717,
 '▁꾸준히': 25991,
 '006': 13942,
 '溥': 31083,
 '▁ڀيرو': 21374,
 '▁kolejnych': 9968,
 '橄': 33792,
 'ッド': 53487,
 '貉': 58636,
 '亮': 16094,
 'ASA': 35860,
 '쾰': 31672,
 '하는데': 55471,
 '▁styre': 47917,
 '▁평균': 50231,
 'יקס': 4158,
 '鹌': 33230,
 '▁Blackberry': 26492,
 '艳': 57668,
 'ႄ': 30496,
 'طالع': 24940,
 'books': 52586,
 '系统': 53508,
 'riji': 54595,
 'ंद्र': 12458,
 '▁(22': 43729,
 'wasser': 51427,
 '번째': 9886,
 '▁Sinne': 49895,
 'ounce': 53541,
 'im': 41736,
 '細緻': 

In [24]:
# 统计vocab中语言词表分布
from tqdm import tqdm
lang_dict = dict()
for k,v in tqdm(processor.tokenizer.vocab.items()):
    lang = langid.classify(k)
    if lang[0] not in lang_dict:
        lang_dict[lang[0]] = 0
    lang_dict[lang[0]] += 1

100%|██████████| 58891/58891 [01:35<00:00, 614.14it/s]


In [27]:
lang_dict = dict(sorted(lang_dict.items(), key=lambda x: x[1], reverse=True))
lang_dict

{'en': 16663,
 'zh': 12734,
 'lv': 5748,
 'ja': 3087,
 'ko': 2538,
 'de': 1183,
 'ru': 975,
 'hu': 745,
 'es': 713,
 'ar': 644,
 'pl': 607,
 'th': 530,
 'fr': 521,
 'el': 509,
 'he': 454,
 'fi': 432,
 'fa': 410,
 'tr': 366,
 'ml': 363,
 'bg': 350,
 'cs': 339,
 'kk': 334,
 'it': 315,
 'sv': 308,
 'uk': 288,
 'be': 287,
 'te': 286,
 'mr': 281,
 'si': 279,
 'sr': 272,
 'ka': 264,
 'et': 260,
 'nl': 257,
 'mn': 253,
 'ta': 249,
 'sl': 248,
 'vi': 235,
 'da': 233,
 'kn': 226,
 'lt': 223,
 'hi': 212,
 'sk': 210,
 'am': 187,
 'bn': 187,
 'gu': 184,
 'ro': 178,
 'lo': 174,
 'az': 158,
 'pa': 148,
 'pt': 142,
 'ne': 141,
 'hy': 136,
 'km': 133,
 'or': 133,
 'eu': 118,
 'mt': 101,
 'rw': 94,
 'id': 91,
 'ur': 89,
 'is': 89,
 'no': 79,
 'sq': 79,
 'ga': 78,
 'ug': 73,
 'hr': 62,
 'mk': 59,
 'gl': 59,
 'ku': 53,
 'vo': 44,
 'oc': 41,
 'ms': 36,
 'bs': 31,
 'af': 29,
 'ps': 28,
 'cy': 24,
 'mg': 22,
 'ht': 20,
 'tl': 16,
 'eo': 15,
 'nn': 14,
 'xh': 13,
 'ca': 13,
 'qu': 12,
 'nb': 12,
 'dz': 11,
 

In [26]:
(lang_dict['en'] + lang_dict['zh'])/len(processor.tokenizer.vocab)

0.49917644461802313

分析一下embed_tokens 和 lm_head

In [30]:
embed_tokens_layer = donut_base_model.decoder.model.decoder.embed_tokens
embed_tokens_layer

Embedding(58891, 1024, padding_idx=1)

In [32]:
embed_tokens_layer.parameters(), type(embed_tokens_layer.parameters())

(<generator object Module.parameters at 0x0000015E0E179E40>, generator)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 embed_tokens_layer[0]                                                                        │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: 'Embedding' object is not subscriptable